<h1>Báo cáo đồ án cuối kì lập trình song song: Song song hóa thuật toán Alpha-Beta Pruning và áp dụng chúng vào game cờ vua</h1>
<h2>Thành viên nhóm</h2>

*   19127503 - Ngô Quốc Phát
*   19127230 - Nguyễn Trí Nhân

<h2>1. Giới thiệu thuật toán</h2>
<p>Alpha-beta pruning là một thuật toán tìm kiếm nhằm giảm số lượng các nút cần được đánh giá bởi thuật toán minimax trong cây tìm kiếm của nó.</p>

<p>Thuật toán dừng việc đánh giá một nước đi khi ít nhất một khả năng đã được tìm thấy chứng minh rằng nước đi đó tồi hơn một nước đi đã được kiểm tra trước đó. Những nước đi như vậy không cần được đánh giá thêm nữa. Khi được áp dụng cho một cây minimax chuẩn, nó trả về cùng một nước đi như minimax, nhưng loại bỏ những nhánh không thể ảnh hưởng đến quyết định cuối cùng.</p>

<p>Thuật toán này thường được sử dụng trong các trò chơi đối kháng như cờ caro, cờ vua, tictactoe,...</p>

*  Input: 1 mảng là 1 chuỗi số nguyên.
*  Output: 1 số tối ưu nhất trong chuỗi số đó.

Ví dụ:

*  Input: A = {2,3,5,9,0,1,7,5}
*  Output: Số tối ưu nhất trong A là: 3

<p>Đồ án này nhóm sẽ thực hiện song song hóa thuật toán Alpha-beta pruning bằng cách viết lại thuật toán mà không sử dụng thư viện của nó, sau đó áp dụng kĩ thuật song song hóa để tối ưu và cải thiện hiệu suất cho thuật toán.</p>


<h2>2. Giới thiệu game cờ vua</h2>
<p>Cờ vua là 1 trò chơi board game dành cho 2 người dựa thuần túy vào chiến thuật và chiến lược.</p>
<p>Cờ vua là một trong những trò chơi trí tuệ phổ biến nhất thế giới.</p>
<p>Nhóm em sẽ áp dụng song song hóa thuật toán Alpha-Beta Pruning vào game này </p>

*   Input: 1 bàn cờ vua sau khi mình đã chọn nước đi.
*   Output: Nước đi tối ưu của game sau khi thực hiện thuật toán.

<h2>3. Ý tưởng</h2>
<p>Nhóm em sẽ sử dụng Google Colab để chạy thuật toán song song bằng cả 3 phương pháp là chạy tuần tự, chạy song song qua CPU và chạy song song qua GPU CUDA </p>


*   Song song đa lõi: Google Colab cung cấp quyền truy cập vào các máy có nhiều lõi CPU với các thư viện Python như multiprocessing hoặc joblib để phân phối các tác vụ tính toán chuyên sâu trên nhiều lõi. Điều này có thể hữu ích để song song hóa các giai đoạn nhất định trong ứng dụng.
*   Tính song song của GPU: Google Colab cung cấp quyền truy cập vào GPU, cụ thể là GPU NVIDIA Tesla. GPU vượt trội ở khả năng xử lý song song và có thể tăng tốc đáng kể các tác vụ học sâu, bao gồm trích xuất tính năng và đào tạo các mô hình máy học. Có thể sử dụng các khung học sâu phổ biến như TensorFlow hoặc PyTorch, được tích hợp hỗ trợ GPU, để tận dụng sức mạnh tính toán song song của GPU.

<p>Ngoài ra, trước khi xử lý bằng thuật toán Alpha-Beta Pruning còn có 2 phương pháp tiền xử lý nhằm mục đích tăng tốc song song hóa thuật toán là: sắp xếp lại và phương pháp Beam Search. </p>
<p>Tuy nhiên, nhóm em sẽ chọn phương pháp sắp xếp lại vì độ chính xác cao hơn, tối ưu hơn phương pháp còn lại. </p>


<h2>4. Phương pháp</h2>
<h3>a. Cài đặt module</h3>

In [ ]:
import math
import numpy as np
from numba import cuda, float32, njit, prange
import timeit

<h3>b. Hàm tạo số ngẫu nhiên sử dụng song song hóa thuật toán Mersenne Twister</h3>
<p>Bước 1: Thiết lập giá trị seed bất kì và mảng result là chuỗi các số ngẫu nhiên</p>
<p>Bước 2: Xác định số lượng threads và blocks cần thiết để chạy kernel.</p>
<p>Bước 3: Gọi hàm mersenne_twister_kernel với số lượng blocks và threads đã xác định, n là số giá trị trong mảng, minimum và maximum lần lượt là giá trị nhỏ nhất và lớn nhất trong chuỗi ngẫu nhiên cần tạo.</p>
<p>Bước 4: Trong hàm mersenne_twister_kernel:</p>

*   Hàm này sử dụng bằng @cuda.jit, cho phép nó chạy song song trên GPU.
*   Sử dụng một mảng mt để lưu trữ trạng thái của Mersenne Twister.
*   Khởi tạo mảng mt với giá trị seed ban đầu.
*   Sử dụng vòng lặp để cập nhật trạng thái của Mersenne Twister và tạo ra các số ngẫu nhiên.
*   Lưu trữ kết quả vào mảng result.

<p>Bước 5: Trả về mảng result chứa các số ngẫu nhiên đã được tạo ra.</p>




In [ ]:
@cuda.jit
def mersenne_twister_kernel(seed, n, minimum, maximum, result):
    idx = cuda.grid(1)
    if idx < n:
        mt = cuda.local.array(624, dtype=np.uint32)
        mt[0] = seed
        for i in range(1, 624):
            mt[i] = 0xFFFFFFFF & (1812433253 * (mt[i - 1] ^ (mt[i - 1] >> 30)) + i)

        index = 624
        for _ in range(idx + 1):
            if index >= 624:
                for i in range(624):
                    y = (mt[i] & 0x80000000) + (mt[(i + 1) % 624] & 0x7FFFFFFF)
                    mt[i] = mt[(i + 397) % 624] ^ (y >> 1)
                    if y % 2 != 0:
                        mt[i] ^= 0x9908B0DF
                index = 0

            y = mt[index]
            y ^= (y >> 11)
            y ^= (y << 7) & 0x9D2C5680
            y ^= (y << 15) & 0xEFC60000
            y ^= (y >> 18)
            index += 1

        result[idx] = minimum + (y % (maximum - minimum + 1))

def random_array(n, minimum, maximum):
    seed = 5489
    result = np.zeros(n, dtype=np.float32)
    threads_per_block = 256
    blocks_per_grid = (n + (threads_per_block - 1)) // threads_per_block
    mersenne_twister_kernel[blocks_per_grid, threads_per_block](seed, n, minimum, maximum, result)
    return result

<h3>c. Ghi và đọc file chứa các chuỗi số ngẫu nhiên</h3>

In [ ]:
def writefile(name, n, minimum, maximum):
    arr = np.array(random_array(n, minimum, maximum),dtype=np.float32)
    arr.tofile(name)

def readfile(name):
    arr_from_file = np.fromfile(name, dtype=np.float32)
    return arr_from_file

<h3> d. Song song hóa qua GPU CUDA thuật toán Quick Sort </h3>
<p>Bước 1: Chuyển mảng arr từ CPU sang GPU bằng cuda.to_device</p>
<p>Bước 2: Xác định số lượng threads và blocks cần thiết để chạy kernel.</p>
<p>Bước 3: Gọi hàm quicksort_kernel với số lượng blocks và threads đã xác định.</p>
<p>Bước 4: Trong hàm quicksort_kernel:</p>

*   Hàm này sử dụng bằng @cuda.jit, cho phép nó chạy song song trên GPU.
*   Sử dụng một mảng stack để lưu trữ các chỉ số của các phân đoạn cần sắp xếp.
*   Sử dụng vòng lặp while để thực hiện sắp xếp các phân đoạn bằng cách chọn một phần tử làm pivot và sắp xếp các phần tử nhỏ hơn pivot sang bên trái và lớn hơn pivot sang bên phải.
*   Sau khi sắp xếp xong một phân đoạn, các phân đoạn con sẽ được đẩy vào stack để tiếp tục sắp xếp.

<p>Bước 5: Sao chép mảng đã sắp xếp từ GPU về lại host CPU bằng d_arr.copy_to_host.</p>

In [ ]:
@cuda.jit
def quicksort_kernel(arr, left, right):
    stack = cuda.local.array(1024, dtype=np.int32)
    top = -1

    top += 1
    stack[top] = left
    top += 1
    stack[top] = right

    while top >= 0:
        right = stack[top]
        top -= 1
        left = stack[top]
        top -= 1

        i = left - 1
        pivot = arr[right]

        for j in range(left, right):
            if arr[j] >= pivot:  # Change comparison to >= for descending order
                i += 1
                arr[i], arr[j] = arr[j], arr[i]

        arr[i + 1], arr[right] = arr[right], arr[i + 1]
        p = i + 1

        if p - 1 > left:
            top += 1
            stack[top] = left
            top += 1
            stack[top] = p - 1

        if p + 1 < right:
            top += 1
            stack[top] = p + 1
            top += 1
            stack[top] = right

def quicksort(arr):
    n = len(arr)
    d_arr = cuda.to_device(arr)
    threads_per_block = 256
    blocks_per_grid = (n + (threads_per_block - 1)) // threads_per_block

    quicksort_kernel[blocks_per_grid, threads_per_block](arr, 0, n-1)
    d_arr.copy_to_host(arr)


**e. Thuật toán Alpha-Beta Pruning:**

Bước 1. **Khởi tạo Ngăn xếp:**
   - Đầu tiên, chúng ta tạo một ngăn xếp để lưu trữ trạng thái của các nút trong cây tìm kiếm.
   - Sau đó, đẩy trạng thái ban đầu vào ngăn xếp với các thông số: độ sâu là 0, chỉ số là 0, người chơi tối đa hóa là True, alpha là âm vô cực, và beta là dương vô cực.

Bước 2. **Lặp lại Khi Ngăn xếp Không Rỗng:**
   - Lấy trạng thái trên cùng ra khỏi ngăn xếp.
   - Nếu độ sâu hiện tại bằng độ sâu tối đa, trả về giá trị tại chỉ số hiện tại.
   - Nếu không, xác định xem vị trí hiện tại là tối đa hóa hay tối thiểu hóa.

Bước 3. **Đối với vị trí tối đa hóa:**
   - Khởi tạo giá trị tối ưu là âm vô cực.
   - Đẩy các nút con phải và trái vào ngăn xếp.
   - Đánh giá các nút con và cập nhật giá trị tối ưu và alpha.
   - Nếu alpha lớn hơn hoặc bằng beta, cắt tỉa các nút còn lại.

Bước 4. **Đối với vị trí tối thiểu hóa:**
   - Khởi tạo giá trị tối ưu là dương vô cực.
   - Đẩy các nút con phải và trái vào ngăn xếp.
   - Đánh giá các nút con và cập nhật giá trị tối ưu và beta.
   - Nếu beta nhỏ hơn hoặc bằng alpha, cắt tỉa các nút còn lại.

In [ ]:
def Alpha_Beta_Pruning_Sequential(values, max_depth):
    stack = np.zeros((1000, 5), dtype=np.float32)
    stack_size = 0

    # Initialize stack
    stack[stack_size, 0] = 0  # depth
    stack[stack_size, 1] = 0  # index
    stack[stack_size, 2] = 1  # maximizingPlayer (True as 1)
    stack[stack_size, 3] = float('-inf')  # alpha
    stack[stack_size, 4] = float('inf')  # beta
    stack_size += 1

    while stack_size > 0:
        stack_size -= 1
        depth = stack[stack_size, 0]
        index = stack[stack_size, 1]
        maximizingPlayer = stack[stack_size, 2]
        alpha = stack[stack_size, 3]
        beta = stack[stack_size, 4]

        if depth == max_depth:
            return values[int(index)]
        else:
            if maximizingPlayer:
                optimum = float('-inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 0  # False as 0
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1

                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = max(optimum, val)
                    alpha = max(alpha, optimum)
                    if beta <= alpha:
                        break
                return optimum
            else:
                optimum = float('inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 1  # True as 1
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1

                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = min(optimum, val)
                    beta = min(beta, optimum)
                    if beta <= alpha:
                        break
                return optimum

In [ ]:
@njit(fastmath=True, cache=True, parallel=True)
def Alpha_Beta_Pruning_CPU(values, max_depth):
    stack = np.zeros((1000, 5), dtype=np.float32)
    stack_size = 0

    # Initialize stack
    stack[stack_size, 0] = 0  # depth
    stack[stack_size, 1] = 0  # index
    stack[stack_size, 2] = 1  # maximizingPlayer (True as 1)
    stack[stack_size, 3] = float('-inf')  # alpha
    stack[stack_size, 4] = float('inf')  # beta
    stack_size += 1

    while stack_size > 0:
        stack_size -= 1
        depth = stack[stack_size, 0]
        index = stack[stack_size, 1]
        maximizingPlayer = stack[stack_size, 2]
        alpha = stack[stack_size, 3]
        beta = stack[stack_size, 4]

        if depth == max_depth:
            result = values[int(index)]
        else:
            if maximizingPlayer:
                optimum = float('-inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 0  # False as 0
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1

                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = max(optimum, val)
                    alpha = max(alpha, optimum)
                    if beta <= alpha:
                        break
                result = optimum
            else:
                optimum = float('inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 1  # True as 1
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1
                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = min(optimum, val)
                    beta = min(beta, optimum)
                    if beta <= alpha:
                        break
                result = optimum
    return result

In [ ]:

@cuda.jit
def Alpha_Beta_Pruning_CUDA(values, max_depth, result):
    stack = cuda.local.array((1000, 5), dtype=float32)
    stack_size = 0

    # Initialize stack
    stack[stack_size, 0] = 0  # depth
    stack[stack_size, 1] = 0  # index
    stack[stack_size, 2] = 1  # maximizingPlayer (True as 1)
    stack[stack_size, 3] = float('-inf')  # alpha
    stack[stack_size, 4] = float('inf')  # beta
    stack_size += 1

    while stack_size > 0:
        stack_size -= 1
        depth = stack[stack_size, 0]
        index = stack[stack_size, 1]
        maximizingPlayer = stack[stack_size, 2]
        alpha = stack[stack_size, 3]
        beta = stack[stack_size, 4]

        if depth == max_depth:
            result[0] = values[int(index)]
        else:
            if maximizingPlayer:
                optimum = float('-inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 0  # False as 0
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1

                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = max(optimum, val)
                    alpha = max(alpha, optimum)
                    if beta <= alpha:
                        break
                result[0] = optimum
            else:
                optimum = float('inf')
                for i in range(1, -1, -1):  # Push right child first
                    stack[stack_size, 0] = depth + 1
                    stack[stack_size, 1] = index * 2 + i
                    stack[stack_size, 2] = 1  # True as 1
                    stack[stack_size, 3] = alpha
                    stack[stack_size, 4] = beta
                    stack_size += 1

                while stack_size > 0 and stack[stack_size - 1, 0] == depth + 1:
                    stack_size -= 1
                    idx = stack[stack_size, 1]
                    val = values[int(idx)]
                    optimum = min(optimum, val)
                    beta = min(beta, optimum)
                    if beta <= alpha:
                        break
                result[0] = optimum




In [ ]:
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[0]
        left = [x for x in arr[1:] if x < pivot]
        right = [x for x in arr[1:] if x >= pivot]
        return quicksort(left) + [pivot] + quicksort(right)


In [ ]:
# Data
writefile('number_data.bin',1000000,1,100000)
values = readfile('number_data.bin')

# values = np.array([911, 940, 907, 259, 560, 230, 952, 848, 347, 730, 973, 864, 828, 164, 958, 314, 842, 997, 655, 996, 963, 2, 459, 115, 725, 799, 152, 361, 698, 833, 318, 215, 911, 580, 276, 922, 937, 886, 148, 931, 114, 96, 174, 114, 194, 322, 773, 60, 225, 517, 123, 102, 911, 36, 236, 764, 459, 981, 23, 731, 880, 268, 622, 692, 215, 497, 69, 135, 759, 250, 912, 510, 312, 418, 467, 142, 990, 827, 197, 153, 91, 349, 46, 493, 551, 573, 213, 818, 475, 215, 921, 570, 998, 73, 453, 742, 957, 376, 206, 173], dtype=np.float32)
max_depth = int(math.log(len(values), 2))
result = np.zeros(1, dtype=np.float32)


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [ ]:

#Chạy tuần tự
print("Chạy tuần tự:")
start = timeit.default_timer()
result = Alpha_Beta_Pruning_Sequential(values, max_depth)
stop = timeit.default_timer()
print("Trước khi sắp xếp dữ liệu:")
print("Result:", result)
print('Time: ', stop - start)
v1=quicksort(values)
start = timeit.default_timer()
result = Alpha_Beta_Pruning_Sequential(values, max_depth)
stop = timeit.default_timer()
print("Sau khi sắp xếp dữ liệu:")
print("Result:", result)
print('Time: ', stop - start)

#Chạy song song trên CPU
print("\nChạy song song trên CPU:")
start = timeit.default_timer()
result_CPU = Alpha_Beta_Pruning_CPU(values, max_depth)
stop = timeit.default_timer()
print("Trước khi sắp xếp dữ liệu:")
print("Result:", result_CPU)
print('Time: ', stop - start)
v2=quicksort(values)
start = timeit.default_timer()
result_CPU = Alpha_Beta_Pruning_CPU(values, max_depth)
stop = timeit.default_timer()
print("Sau khi sắp xếp dữ liệu:")
print("Result:", result_CPU)
print('Time: ', stop - start)

#Chạy song song trên GPU
result_GPU = np.zeros(1, dtype=np.float32)
v3 =  np.array(quicksort(values))

# Launch kernel with more threads and blocks
threads_per_block = 256
blocks_per_grid = (values.size + (threads_per_block - 1)) // threads_per_block
start = timeit.default_timer()
Alpha_Beta_Pruning_CUDA[blocks_per_grid, threads_per_block](values, max_depth, result_GPU)
stop = timeit.default_timer()
print("\nChạy song song trên GPU CUDA:")
print("Trước khi sắp xếp dữ liệu:")
print("Result:", result_GPU[0])
print('Time: ', stop - start)

result_GPU_2 = np.zeros(1, dtype=np.float32)
threads_per_block = 256
blocks_per_grid = (values.size + (threads_per_block - 1)) // threads_per_block
start = timeit.default_timer()
Alpha_Beta_Pruning_CUDA[blocks_per_grid, threads_per_block](v3, max_depth, result_GPU_2)
stop = timeit.default_timer()
print("Sau khi sắp xếp dữ liệu:")
print("Result:", result_GPU_2[0])
print('Time: ', stop - start)

Chạy tuần tự:
Trước khi sắp xếp dữ liệu:
Result: 69303.0
Time:  0.00026006299958680756
Sau khi sắp xếp dữ liệu:
Result: 69303.0
Time:  0.00033218699991266476

Chạy song song trên CPU:
Trước khi sắp xếp dữ liệu:
Result: 69303.0
Time:  0.47552021499996044
Sau khi sắp xếp dữ liệu:
Result: 69303.0
Time:  0.00021553199985646643


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:888: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))



Chạy song song trên GPU CUDA:
Trước khi sắp xếp dữ liệu:
Result: 69303.0
Time:  1.0776357500003542
Sau khi sắp xếp dữ liệu:
Result: 1.0
Time:  0.10346668700003647


In [ ]:
v3

array([1.e+00, 1.e+00, 1.e+00, ..., 1.e+05, 1.e+05, 1.e+05], dtype=float32)

<h2>5. Thành tựu</h2>

*   100%: Xây dựng được thuật toán ở những cách chạy tuần tự, chạy song song bằng CPU và chạy song song bằng GPU một cách hoàn thiện nhất. Mục tiêu ở bước này sẽ là cho ra một cái nhìn khách quan về hiệu suất của từng cách làm. (Đã hoàn thành)
*   125%: Xây dựng được trò chơi cờ vua theo kiểu tuần tự nhưng chưa thể song song hóa trò chơi này qua CPU, GPU và shared memory. Bị lỗi game khi chơi lượt 3.


<h2>6. Khó khăn</h2>

*   Tài liệu hạn chế.
*   Chưa chạy được Shared Memory.
*   Chạy song song CPU, GPU và Shared memory bị lỗi khi áp dụng với game cờ vua.
*   Thời gian chạy tuần tự mất nhiều thời gian.
*   Cả 2 thành viên nhóm mình bị dí deadline từ nơi làm việc nên trễ hẹn nộp seminar 2.

<h2>7. References</h2>
<p>[1] https://github.com/njmarko/alpha-beta-pruning-minmax-checkers</p>
<p>[2] https://www.researchgate.net/publication/343945419_IMPLEMENTATION_OF_SEQUENTIAL_AND_PARALLEL_ALPHA-BETA_PRUNING_ALGORITHM</p>
<p>[3] https://arxiv.org/pdf/1908.11660</p>
<p>[4] https://www.kaggle.com/code/garyongguanjie/mini-max-alpha-beta-no-np/<p>
<p>[5] https://tonypoer.io/2016/10/28/implementing-minimax-and-alpha-beta-pruning-using-python/</p>
<p>[6] https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning</p>
<p>[7] https://medium.com/chat-gpt-now-writes-all-my-articles/advanced-ai-alpha-beta-pruning-in-python-through-tic-tac-toe-70bb0b15db05</p>
<p>[8] https://www.mygreatlearning.com/blog/alpha-beta-pruning-in-ai/</p>
<p>[9] https://en.wikipedia.org/wiki/Mersenne_Twister</p>
<p>[10] https://github.com/yinengy/Mersenne-Twister-in-Python</p>



